In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from import_ipynb import * 
import Bert_model 
import collections
import json 
import random 
import re 

import six 
import tensorflow as tf 

In [ ]:
# Xây dựng một bert thử nghiệm từ model Bert đã tạo trước đó 
class BertModelTest(tf.test.TestCase):
    
    # xây dựng một class là Bert model tester trong model test 
    class BertModelTester(object):

        # thiết lập một phương thức khởi tạo các tham số cho đối tượng 
        def __init__(self, parent , batch_size =13, seq_length=7,
                is_training=True, use_input_mask=True, 
                use_token_type_ids=True, vocab_size=99,
                hidden_size =32, num_hidden_layers =5, 
                num_attention_heads = 4, hidden_act= "gelu",
                hidden_dropout_prob=0.1, intermediate_size =37,
                attention_probs_dropout_prob=0.1,
                max_position_embeddings=512,
                type_vocab_size=16,
                initializer_range=0.02,
                scope=None):
            # định nghĩa các thuộc tính được sử dụng cho mô hình 
            self.parent = parent
            self.batch_size = batch_size
            self.seq_length = seq_length
            self.is_training = is_training
            self.use_input_mask = use_input_mask
            self.use_token_type_ids = use_token_type_ids
            self.vocab_size = vocab_size
            self.hidden_size = hidden_size
            self.num_hidden_layers = num_hidden_layers
            self.num_attention_heads = num_attention_heads
            self.intermediate_size = intermediate_size
            self.hidden_act = hidden_act
            self.hidden_dropout_prob = hidden_dropout_prob
            self.attention_probs_dropout_prob = attention_probs_dropout_prob
            self.max_position_embeddings = max_position_embeddings
            self.type_vocab_size = type_vocab_size
            self.initializer_range = initializer_range
            self.scope = scope

        # xây dựng phương thức tạo mô hình Bertester 
        def create_model(self):
            # khởi tạo một biến input_ids gán nó bằng kết quả của BertModelTest.ids_tensor 
            # ids_tensor là 1 phương thức được sử dụng để tạo một tensor ngẫu nhiên 
            # có hình dnagj trong kích thước voab_size 
            input_ids = BertModelTest.ids_tensor([self.batch_size, self.seq_length], self.vocab_size)

            # khởi tạo một biến là input_mask gán = None 
            input_mask = None 
            # và kiểm tra xem có tồn tại giá trị use_token_type_ids không 
            if self.use_input_mask: 
                # nếu có ta tạo một biến là input_mask 
                # gán nó vào kết qura của một tensor được tạo ngẫu nhiên bởi modeltest với vocab_size = 2
                input_mask = BertModelTest.ids_tensor([self.batch_size, self.seq_length], vocab_size=2)

            # tươnh tự như trên ta khởi tạo một giá trị tokne_tyoe_ids gán = NOne 
            token_type_ids = None 
            # và kiểm tra xem giá trị token_type_ids có tồn tại 
            if self.use_token_type_ids:
                # nêis có ta tạo gan cho token_tyoe_ids bằng với gia strij của 1 tensor được tạo ngẫu nhiên
                # bởi modeltest  vơi kích thước vocab_size 
                token_type_ids = BertModelTest.ids_tensor([self.batch_size, self.seq_length], self.vocab_size)

            # định cấu hình các tham số được sử dụng cho mô hình 
            config = Bert_model.BertConfig(
                # kích thước bộ từ điển 
                vocab_size=self.vocab_size,
                # trạng thái ẩn 
                hidden_size=self.hidden_size,
                # số lớp ẩn 
                num_hidden_layers=self.num_hidden_layers,
                # đầu chú ý 
                num_attention_heads=self.num_attention_heads,
                # số chiều lớp ffn 
                intermediate_size=self.intermediate_size,
                # activations lớp ẩn 
                hidden_act=self.hidden_act,
                # dropout cho hidden layers
                hidden_dropout_prob=self.hidden_dropout_prob,
                # dropout _attention 
                attention_probs_dropout_prob=self.attention_probs_dropout_prob,
                # max position for embeddings 
                max_position_embeddings=self.max_position_embeddings,
                # type of vocab_size 
                type_vocab_size=self.type_vocab_size,
                # weight initializer 
                initializer_range=self.initializer_range
            )

            # định dạng mô hình 
            model = Bert_model.BertModel(
                config=config,
                is_training=self.is_training,
                input_ids=input_ids,
                input_mask=input_mask,
                token_type_ids=token_type_ids,
                scope=self.scope
            )

            # định dạng các tham số đầu ra cho mô hình 
            outputs = {
                "embedding_output": model.get_embedding_output(),
                "sequence_output": model.get_sequence_output(),
                "pooled_output": model.get_pooled_output(),
                "all_encoder_layers": model.get_all_encoder_layers(),
                }
            return outputs

        # xây dựng một phương thức kiểm tra kết quả của các tensor đầu ra 
        def check_output(self, result):
            # thực hiện kiểm tra xem kết quả của tensor embedding_output.shape
            # có giống như hình dạng đầu ra mong muốn 
            self.parent.assertAllEqual(
                result['embedding_output'].shape , 
                [self.batch_size, self.seq_length, self.hidden_size]
            )
            # tương tự thực hiện kiểm tra xem kết quả của tensor sequence_length.shape
            # có giống như hình dạng đầu ra mong muốn 
            self.parent.assertAllEqual(
                result['sequence_output'].shape , 
                [self.batch_size, self.seq_length, self.hidden_size]
            )
            # tương tự thực hiện kiểm tra xem kết quả của tensor epooled_output.shape
            # có giống như hình dạng đầu ra mong muốn 
            self.parent.assertAllEqual(result["pooled_output"].shape,
                                 [self.batch_size, self.hidden_size])

    # xây dựng phương thức test_config_to_json_string 
    # dùng để kiểm tra đơn vị cho lớp modeling.BertModel.Bertconfig 
    def test_config_to_json_string(self):
        # tạo một đối tượng của lớp Bert_model.Bertconfig với tham số vocab và hidden_size 
        config = Bert_model.BertConfig(vocab_size=99, hidden_size=37)
        # gọi phương thức config.to_json_string dể chuyển một đối tượng config thành một chuỗi json
        # Sử dụng hàm json.loads() để phân tích chuỗi JSON thành một đối tượng Python, được lưu trữ trong biến obj.
        obj = json.loads(config.to_json_string())
        # kiểm tra xem giá trị của obj[“vocab_size”] và obj[“hidden_size”] 
        # có bằng với giá trị ban đầu của config hay không
        self.assertEqual(obj["vocab_size"], 99)
        self.assertEqual(obj["hidden_size"], 37)

    # xây dựn một phương thức chạy thử nghiệm nhận tham số chuyền vào là tester 
    # đây là một đối tượng create_model và check output
    def run_tester(self, tester):
        # tạo một đối tượng test_session tạo ra 1 phiên bản tensorflow cho mục đích thử nghiễm 
        with self.test_session() as sess: 
            # gọi phương thức tester.create_model trả về một phương thức thao tác xác định mô hình 
            # cần kiểm tra 
            ops = tester.create_model()
            # tạo một biến operation sử dụng tf.group để nhóm 2 operation con 
            # sử dụng để lưu trữ và cập nhật và lưu trữ các giá trị trong huấn luyện 
            # và dự đoán 
            init_op = tf.group(tf.global_variables_initializer(),
                         tf.local_variables_initializer())

            # chạy opreation với sess.run 
            sess.run(init_op)
            # chạy mô hình để có được các kết quả huấn luyện và lưu nó vapf ouput 
            output_result = sess.run(ops)
            # gọi phương thức checkpoint hương thức này lấy tham số out_result và xác minh rằng 
            # nó khớp với kết quả đầu ra dự kiến ​​của mô hình
            tester.check_output(output_result)
            # gọi phương thức self.assert_all_tensors_reachable, phương thức này lấy sess 
            # và danh sách các phép toán làm tham số, đồng thời 
            # kiểm tra xem tất cả các tensor trong biểu đồ có thể truy cập được từ các phép toán hay không
            self.assert_all_tensors_reachable(sess, [init_op, ops])


    # Xây dựng một phương thức tạo ra các tensor ngẫu nhiênn dtype = int32 
    # và có hình dạng trong kích thước vocab_size 
    @classmethod
    def ids_tensor(cls, shape, vocab_size, rng=None, name=None):
        """Creates a random int32 tensor of the shape within the vocab size."""
        # kiểm tra xem ngr có = None if = NONE đặt nó  = 1 xác xuất ngẫu nhiên
        if rng is None:
            rng = random.Random()

        # tạo mộ biến total dim  = 1
        total_dims = 1
        # duyệt qua số chiều của tensor lấy ra giá trị tưgf chiều 
        for dim in shape:
            # nhân nó với 1 
            total_dims *= dim

        # tạo 1 danh sách để lưu trữ các giá trị của tensor 
        values = []
        # duyệt qua 1 danh 0 -> total-dim
        for _ in range(total_dims):
            # tạo ra 1 tensor và thêm nó vào dnah sách 
            values.append(rng.randint(0, vocab_size - 1))
        # trả về 1 tensor với hình dnagj mong đợi 
        return tf.constant(value=values, dtype=tf.int32, shape=shape, name=name)
    
    # 
    def assert_all_tensors_reachable(self, sess, outputs):
        """Checks that all the tensors in the graph are reachable from outputs."""
        # Phần này là phần lọc ra các toán tử không thể truy cập được mà không nằm trong danh sách bỏ qua. 
        # đầu tiên tạo ra một đồ thị grahp từ sess
        graph = sess.graph

        # sau đó tạo ra một danh sách các cuỗi ignore  chứa các biêyr thức chính quy để 
        # bỏ qua các toán tử có tên khớp với chúng .
        ignore_strings = [
            "^.*/assert_less_equal/.*$",
            "^.*/dilation_rate$",
            "^.*/Tensordot/concat$",
            "^.*/Tensordot/concat/axis$",
            "^testing/.*$",
        ]

        # sau đó tạo ra  danh sách ácc biểu thức chính quy từ chuỗi ignore 
        ignore_regexes = [re.compile(x) for x in ignore_strings]

        # gọi phương thức get_unreachable để lấy ra các toán tử không thể truy cập được 
        # từ đầu ra outputs và trả về  danh scahs ko thể truy cập 
        unreachable = self.get_unreachable_ops(graph, outputs)
        # tiếp theo tạo  danh sách rỗng để lưu trữ các phần tử không thể truy cập 
        filtered_unreachable = []
        # duyệt qua dnah sách các phần tử không thể truy cập 
        for x in unreachable:
            # tạo ra  giá trị bool là do_ignore để quyêt định xem có bỏ qua nó 
            # hay không : Gán = False
            do_ignore = False
            # duyệt qua các biểu thức chính quy trong ignore_regex
            for r in ignore_regexes:
                # kiểm tra xem tên toán tử có trùng bới tên biểu thức không
                m = r.match(x.name)
                # nếu không trùng 
                if m is not None:
                    # thì gán ignore = True tức bỏ qua
                    do_ignore = True
            # và tiếp tục lặp
            if do_ignore:
                continue
            # trường hơpk do_ignore=  False nghĩa là trùng tên với biểu thức và thêm nó avof danh sách 
            # loại bỏ
            filtered_unreachable.append(x)
            # . Cuối cùng, hàm này gán lại unreachable bằng filtered_unreachable,
            #  để lấy ra các toán tử không thể truy cập được sau khi đã lọc.
        unreachable = filtered_unreachable
        # kiểm tra xem có toán tử nào không thể truy cập được hay không
        # Hàm này sử dụng phương thức join để nối các tên của các toán tử không thể truy cập được
        # thành một chuỗi, cách nhau bởi dấu cách.
        self.assertEqual(
            len(unreachable), 0, "The following ops are unreachable: %s" %
            (" ".join([x.name for x in unreachable])))

    @classmethod
    def get_unreachable_ops(cls, graph, outputs):
        """Finds all of the tensors in graph that are unreachable from outputs.
            Tìm tất cả các tensor trong biểu đồ không thể truy cập từ các đầu ra.
                
            """
        # đầu tiên làm phẳng danh sách các dầu ra với flatten_recursive()
        outputs = cls.flatten_recursive(outputs)
        # sau đó tạo ra 3 từ điển để lưu trữ các quan hệ và các toán tử
        output_to_op = collections.defaultdict(list)
        op_to_all = collections.defaultdict(list)
        assign_out_to_in = collections.defaultdict(list)

        # duyệt qua các toán tử trong đồ thị 
        for op in graph.get_operations():
            # và thêm tên các tensor đầu vào , đầu ra của mỗi toán tử 
            # vào các từ điển tương ứng 
            for x in op.inputs:
                op_to_all[op.name].append(x.name)
            for y in op.outputs:
                output_to_op[y.name].append(op.name)
                op_to_all[op.name].append(y.name)
            
            # Nếu toán tử là loại “Assign”, hàm này còn thêm tên của các 
            # tensor đầu vào vào từ điển assign_out_to_in với khóa là tên của tensor đầu ra
            if str(op.type) == "Assign":
                for y in op.outputs:
                    for x in op.inputs:
                        assign_out_to_in[y.name].append(x.name)

            # tạo một rừ điển khác assign_groups để lưu trữ nhóm các tensor có quan hệ gán vơi nhau
            assign_groups = collections.defaultdict(list)
            # Hàm này duyệt qua các khóa trong assign_out_to_in, và tạo ra một danh sách các
            # tên tensor (name_group) có quan hệ gán với khóa đó
            for out_name in assign_out_to_in.keys():
                name_group = assign_out_to_in[out_name]
            #Hàm này cũng thêm tên của tensor đầu ra và các tên khác trong name_group vào từ điển 
            # assign_groups với khóa là tên đó. Sau đó, 
            # hàm này tạo ra một từ điển (seen_tensors) để đánh dấu các tensor đã được duyệt qua
            for n1 in name_group:
                assign_groups[n1].append(out_name)
                for n2 in name_group:
                    if n1 != n2:
                        assign_groups[n1].append(n2)
            #  Hàm này thêm tên của các tensor trong outputs vào stack. 
            seen_tensors = {}
            stack = [x.name for x in outputs]
            # Hàm này lặp lại cho đến khi stack rỗng
            while stack:
                name = stack.pop()
            # : lấy ra một tên tensor khỏi stack (name), 
            # kiểm tra xem nó đã được duyệt qua chưa,
            if name in seen_tensors:
                continue
            #  nếu chưa thì đánh dấu nó trong seen_tensors
            seen_tensors[name] = True
            #  kiểm tra xem nó có nằm trong output_to_op không,
            #  nếu có thì duyệt qua các toán tử có liên quan đến nó (op_name)
            if name in output_to_op:
                for op_name in output_to_op[name]:
                    # kiểm tra xem op_name có nằm trong op_to_all không, 
                    # nếu có thì duyệt qua các tensor có liên quan đến op_name (input_name)
                    if op_name in op_to_all:
                        for input_name in op_to_all[op_name]:
                            # và thêm chúng vào stack nếu chưa có trong stack, 
                            # tạo ra một danh sách rỗng (expanded_names) để lưu trữ các tên tensor mở rộng, 
                            if input_name not in stack:
                                stack.append(input_name)

            expanded_names = []
            # iểm tra xem name có nằm trong assign_groups không
            if name in assign_groups:
                # nếu có thì duyệt qua các tensor có quan hệ gán với name (assign_name),
                for assign_name in assign_groups[name]:
                    # và thêm chúng vào expanded_names
                    expanded_names.append(assign_name)
            # duyệt qua các tên trong expanded_names,
            for expanded_name in expanded_names:
                # và thêm chúng vào stack nếu chưa có trong stack
                if expanded_name not in stack:
                    stack.append(expanded_name)
            # Cuối cùng, hàm này tạo ra một danh sách rỗng (unreachable_ops) 
            # để lưu trữ các toán tử không thể truy cập được
            unreachable_ops = []
            # Hàm này duyệt qua các toán tử trong đồ thị, và kiểm tra xem có
            # tensor nào liên quan đến chúng mà không nằm trong seen_tensors không
            for op in graph.get_operations():
                is_unreachable = False
            all_names = [x.name for x in op.inputs] + [x.name for x in op.outputs]

            for name in all_names:
                # kiểm tra xem có tensor nào liên quan đến chúng mà không nằm trong seen_tensors không. 
                if name not in seen_tensors:
                    # Nếu có, hàm này gán biến boolean (is_unreachable) bằng True,
                    is_unreachable = True
            if is_unreachable:
                # à thêm toán tử đó vào unreachable_ops.
                #  Hàm này trả về danh sách các toán tử không thể truy cập được.
                unreachable_ops.append(op)
        return unreachable_ops
    
    @classmethod
    def flatten_recursive(cls, item):
        """Flattens (potentially nested) a tuple/dictionary/list to a list."""
        output = []
        if isinstance(item, list):
            output.extend(item)
        elif isinstance(item, tuple):
            output.extend(list(item))
        elif isinstance(item, dict):
            for (_, v) in six.iteritems(item):
                output.append(v)
        else:
            return [item]

        flat_output = []
        for x in output:
            flat_output.extend(cls.flatten_recursive(x))
        return flat_output


if __name__ == "__main__":
  tf.test.main()

        